In [31]:
1 + 1

2

In [32]:
import os
os.chdir('d://End to End NLP')
%pwd

'd:\\End to End NLP'

In [40]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    tokenizer_path: Path
    metric_file_name: Path

In [41]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [42]:
class ConfigurationManager:
    def __init__(self, config_file_path= CONFIG_FILE_PATH, params_file_path= PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([Path(self.config.artifacts_root)])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([Path(config.root_dir)])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=Path(config.root_dir),
            data_path=Path(config.data_path),
            model_path=Path(config.model_path),
            tokenizer_path=Path(config.tokenizer_path),
            metric_file_name=Path(config.metric_file_name)
        )

        return model_evaluation_config

In [43]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_from_disk
import evaluate
import torch
import pandas as pd
from tqdm import tqdm

In [ ]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def generate_batch_sized_chunks(self, list_of_elements, batch_size):
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    def calculate_metric_on_test_ds(self, dataset, metric, model, tokenizer,
                            batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu",
                            column_text="article",
                            column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):

            inputs = tokenizer(article_batch, max_length=1024,  truncation=True,
                            padding="max_length", return_tensors="pt")

            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device),
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''

            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                    clean_up_tokenization_spaces=True)
                for s in summaries]

            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]

            metric.add_batch(predictions=decoded_summaries, references=target_batch)

        # Return the computed metrics
        return metric.compute()


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)

        data_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
        rouge_metric = evaluate.load("rouge")

        score = self.calculate_metric_on_test_ds(
            dataset=data_samsum_pt["test"][0:1],
            metric=rouge_metric,
            model=model,
            tokenizer=tokenizer,
            batch_size=16,
            device=device,
            column_text="dialogue",
            column_summary="summary"
        )

        rouge_dict = dict((rn, score[rn]) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name, index=False)

In [55]:
config = ConfigurationManager()
model_evaluation_config = config.get_model_evaluation_config()
model_evaluation = ModelEvaluation(config=model_evaluation_config)
model_evaluation.evaluate()

[2025-11-17 17:19:59,168] 15 - INFO - YAML file: config\config.yaml loaded successfully
[2025-11-17 17:19:59,170] 15 - INFO - YAML file: params.yaml loaded successfully
[2025-11-17 17:19:59,171] 27 - INFO - Directory created at: artifacts
[2025-11-17 17:19:59,172] 27 - INFO - Directory created at: artifacts\model_evaluation
[2025-11-17 17:19:59,170] 15 - INFO - YAML file: params.yaml loaded successfully
[2025-11-17 17:19:59,171] 27 - INFO - Directory created at: artifacts
[2025-11-17 17:19:59,172] 27 - INFO - Directory created at: artifacts\model_evaluation


100%|██████████| 1/1 [00:21<00:00, 21.78s/it]

[2025-11-17 17:20:24,776] 83 - INFO - Using default tokenizer.
